# Checking XSHOOTER-NIR BPM

In [1]:
from __future__ import (print_function, absolute_import)
from __future__ import (division, unicode_literals)

# importing

import numpy as np
from astropy.io import fits
from astropy import units
from glob import glob
import os
import argparse

from pypeit import ginga
from pypeit.core import pixels
from pypeit import traceslits
from pypeit import processimages
from pypeit import scienceimage
from pypeit import arcimage
from pypeit.core import arc
from pypeit import wavecalib
from pypeit import wavetilts
from pypeit import waveimage
from pypeit import flatfield
from pypeit import traceimage
from pypeit import specobjs
from pypeit import utils
from pypeit import biasframe

# Spectrgraph and Settings
from pypeit.spectrographs.util import load_spectrograph
from pypeit.par import pypeitpar
from pypeit import ginga
from pypeit.core.skysub import global_skysub
from pypeit.core import skysub
from pypeit.core import extract
from pypeit.core.flux import generate_sensfunc
#from pypeit.core.flux import generate_sensfunc
from linetools import utils as ltu

from IPython import embed
from matplotlib import pyplot as plt

In [2]:
import os
# os.system('ginga --modules=RC&')

## Load Spectrograph

In [3]:
spectro_name = 'vlt_xshooter_nir'
spectrograph = load_spectrograph(spectrograph=spectro_name)

# Detector settings
par = spectrograph.default_pypeit_par()

print(spectrograph.detector)

[Parameter      Value          Default  Type              Callable
-----------------------------------------------------------------
dataext        0              0        int               False   
specaxis       1              0        int               False   
specflip       False          False    bool              False   
spatflip       False          False    bool              False   
xgap           0.0            0.0      int, float        False   
ygap           0.0            0.0      int, float        False   
ysize          1.0            1.0      int, float        False   
platescale     0.197          0.135    int, float        False   
darkcurr       0.0            0.0      int, float        False   
saturation     200000.0       65535.0  int, float        False   
mincounts      -1000.0        -1000.0  int, float        False   
nonlinear      0.86           0.86     int, float        False   
numamplifiers  1              1        int               False   
gain     

## Load Relevant Files

In [4]:
xs_dir = '/home/ema/Dropbox/XshooterTestBed/VLT_XSHOOTER/'

xsnir_dir = xs_dir+'NIR/'

flat_files = [xsnir_dir+'XSHOO.2016-08-03T14:41:06.410.fits.gz',
              xsnir_dir+'XSHOO.2016-08-03T14:41:59.007.fits.gz',
              xsnir_dir+'XSHOO.2016-08-03T14:42:54.280.fits.gz']

bias_files = [xsnir_dir+'XSHOO.2016-08-03T14:41:32.042.fits.gz',
              xsnir_dir+'XSHOO.2016-08-03T14:42:27.310.fits.gz',
              xsnir_dir+'XSHOO.2016-08-03T14:43:22.583.fits.gz']

## Bias

In [5]:
par['calibrations']['biasframe']['number'] = 3

bImage = biasframe.BiasFrame(spectrograph, files=bias_files)

bimage = bImage.process()

[WARNING] :: processimages.py 541 process() - Your images have not been bias subtracted!
[INFO]    :: combine.py 62 comb_frames() - Combining 3 bias frames
[WORK IN ]::
[PROGRESS]:: combine.py 66 comb_frames() - lscomb feature has not been included here yet...
[INFO]    :: combine.py 95 comb_frames() - Finding saturated and non-linear pixels
[INFO]    :: combine.py 119 comb_frames() - Rejecting cosmic rays
[INFO]    :: combine.py 163 comb_frames() - Not rejecting any low/high pixels
[INFO]    :: combine.py 171 comb_frames() - Rejecting deviant pixels
[INFO]    :: combine.py 188 comb_frames() - Combining frames with a weightmean operation
[INFO]    :: combine.py 201 comb_frames() - Replacing completely masked pixels with the maxnonsat value of the input frames
[INFO]    :: combine.py 215 comb_frames() - 3 bias frames combined successfully!


# Flat

In [6]:
tImage = traceimage.TraceImage(spectrograph, files=flat_files)

tflat = tImage.process(bias_subtract=bimage, trim=True)

[INFO]    :: processimages.py 412 bias_subtract() - Bias subtracting your image(s)
[INFO]    :: processimages.py 418 bias_subtract() - Subtracting bias image from raw frame
[INFO]    :: processimages.py 418 bias_subtract() - Subtracting bias image from raw frame
[INFO]    :: processimages.py 418 bias_subtract() - Subtracting bias image from raw frame
[INFO]    :: combine.py 62 comb_frames() - Combining 3 trace_image frames
[WORK IN ]::
[PROGRESS]:: combine.py 66 comb_frames() - lscomb feature has not been included here yet...
[INFO]    :: combine.py 95 comb_frames() - Finding saturated and non-linear pixels
[INFO]    :: combine.py 119 comb_frames() - Rejecting cosmic rays
[INFO]    :: combine.py 163 comb_frames() - Not rejecting any low/high pixels
[INFO]    :: combine.py 171 comb_frames() - Rejecting deviant pixels
[INFO]    :: combine.py 188 comb_frames() - Combining frames with a weightmean operation
[INFO]    :: combine.py 201 comb_frames() - Replacing completely masked pixels with

# Show images on GINGA

In [13]:
bpm = spectrograph.bpm(shape=tflat.shape, det=1)
ginga.show_image(tflat, chname='Flat')
ginga.show_image(bpm, chname='BPM')

(<ginga.util.grc.RemoteClient at 0x7fd8a9baf898>,
 <ginga.util.grc._channel_proxy at 0x7fd8a8a964e0>)

In [10]:
_, _ = ginga.show_image(tflat*(1.-bpm))